In [1]:
%pwd 

'/workspaces/OpenDVCW'

In [2]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import OpenDVCW
import numpy as np
import load
import tensorflow as tf
import matplotlib.pyplot as plt
import DataGen
import Callbacks
import datetime

/workspaces/OpenDVCW


In [7]:
BATCH_SIZE = 1
EPOCHS = 10000
STEPS_PER_EPOCH = 100
Height = 240
Width = 240
Channel = 3
lmbda = 2048
lr_init = 1e-4
early_stop = 7
I_QP=27

args = OpenDVCW.Arguments()
last = 2
checkponts_last_path = "checkpoints_L_{}_{}_{}x{}/".format(lmbda, last, Width, Height)
checkponts_new_path = "checkpoints_L_{}_{}_{}x{}/".format(lmbda, last+1, Width, Height)
save_name = "model_save_L_{}_{}_{}x{}".format(lmbda, last+1, Width, Height)


In [8]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
rm -rf ./logs/

In [10]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [11]:
model = OpenDVCW.OpenDVC(width=Width, height=Height, batch_size=BATCH_SIZE, num_filters=128, lmbda=lmbda)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_init),)
print("* [Model compiled]...")

print("* [Loading dataset]...")
data = DataGen.DataVimeo90kGenerator("folder_cloud_test.npy", 
                                    BATCH_SIZE,
                                    (Height,Width,Channel),
                                    Channel,
                                    True, 
                                    I_QP,
                                    True)

print("Loading weights")
model.load_weights(checkponts_last_path)


2022-02-25 14:42:31.607586: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 14:42:31.726634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 14:42:31.727460: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 14:42:31.729578: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

* [Model compiled]...
* [Loading dataset]...
Loading weights


2022-02-25 14:42:40.199464: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [12]:
# model.layers[0].trainable = False
# model.layers[1].trainable = False
# model.layers[2].trainable = True
# model.layers[3].trainable = True
# model.layers[4].trainable = False
# model.layers[5].trainable = True

In [13]:
for layer in model.layers:
    print(layer.name, layer.trainable)

mv_analysis True
mv_synthesis True
res_analysis True
res_synthesis True
optical_flow True
motion_compensation True


In [14]:
# tf.config.run_functions_eagerly(True)
hist = model.fit(x=data, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE,
                callbacks=[
                    Callbacks.MemoryCallback(),
                    Callbacks.LearningRateReducer(),
                    tf.keras.callbacks.ModelCheckpoint(filepath=checkponts_new_path, save_weights_only=True, save_freq='epoch', monitor="loss", mode='min',  save_best_only=True, verbose=1), 
                    tf.keras.callbacks.TerminateOnNaN(),
                    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=early_stop),
                    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, update_freq="epoch"),            
                    ],
				)

Epoch 1/10000


2022-02-25 14:43:15.406949: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100


100/100 [==============================] - ETA: 0s - loss: 1.4522 - bpp: 0.8513 - mse: 2.9343e-04- ETA: 1s - loss: 1.4468 - bpp: 0.8470 - m[MemoryCallback]:  4730772

Epoch 00001: loss improved from inf to 1.45223, saving model to checkpoints_L_2048_3_240x240/
100/100 [==============================] - 44s 128ms/step - loss: 1.4522 - bpp: 0.8513 - mse: 2.9343e-04
Epoch 2/10000
100/100 [==============================] - ETA: 0s - loss: 1.5437 - bpp: 0.8686 - mse: 3.2964e-04[MemoryCallback]:  4944904

Epoch 00002: loss did not improve from 1.45223
100/100 [==============================] - 12s 118ms/step - loss: 1.5437 - bpp: 0.8686 - mse: 3.2964e-04
Epoch 3/10000
100/100 [==============================] - ETA: 0s - loss: 1.5253 - bpp: 0.8362 - mse: 3.3646e-04[MemoryCallback]:  5046284

Epoch 00003: loss did not improve from 1.45223
100/100 [==============================] - 13s 124ms/step - loss: 1.5253 - bpp: 0.8362 - mse: 3.3646e-04
Epoch 4/10000
100/100 [=============================

In [ ]:
path = load.load_random_path("folder_cloud_test.npy")
i=0
out_bin = "Test_com/test{}.bin".format(i)
out_decom = "Test_com/testdcom{}.png".format(i)
p_on_test = "Test_com/test_p_frame{}.png".format(i)
i_on_test = "Test_com/test_i_frame{}.png".format(i)

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)

OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))

OpenDVCW.compress(model, i_frame, p_frame, out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, out_bin, out_decom, 240, 240)

/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00035/0959/im1.png
compress


In [ ]:
model.save(save_name, save_format="tf")

in the compress
in decompress


INFO:tensorflow:Assets written to: model_save_L_1024_2_240x240/assets


INFO:tensorflow:Assets written to: model_save_L_1024_2_240x240/assets
